In [1]:
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time


driver = webdriver.Chrome()
# driver = webdriver.Edge()
driver.get("https://etfdb.com/etfs/asset-class/equity/?sort_name=assets_under_management&sort_order=desc&page=1#etfs__overview&sort_name=symbol&sort_order=asc&page=1")
time.sleep(5)


In [2]:
soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find("table", id="etfs")

In [3]:
def normalize_header(title: str) -> str:
    # Remove deduplicates
    title = title[:len(title)//2]

    # Remove text within parentheses
    title = re.sub(r"\(.*?\)", "", title)   

    # Normalize formatting
    title = title.lower().replace(" ", "_").replace("-", "_")

    return title.strip().rstrip("_")

In [4]:
cells = table.find('thead').find('tr').find_all('th')
headers = [normalize_header(cell.get_text(strip=True)) for cell in cells]

In [5]:
headers

['symbol',
 'etf_name',
 'total_assets',
 'ytd',
 'avg_volume',
 'previous_closing_price',
 '1_day_change',
 '1_week',
 '1_month',
 '1_year',
 '3_year',
 '5_year',
 'ytd_ff',
 '1_week_ff',
 '4_week_ff',
 '1_year_ff',
 '3_year_ff',
 '5_year_ff',
 'etf_database_category',
 'inception',
 'er',
 'commission_free',
 'annual_dividend_rate',
 'dividend_date',
 'dividend',
 'annual_dividend_yield_%',
 'p/e_ratio',
 'beta',
 '#_of_holdings',
 '%_in_top_10',
 'complete',
 'st_cap_gain_rate',
 'lt_cap_gain_rate',
 'tax_form',
 'lower_bollinger',
 'upper_bollinger',
 'support_1',
 'resistance_1',
 'rsi',
 'advanced',
 'fact_sheet',
 'etf_holdings',
 'chart',
 'etf_home_page',
 'head_to_head',
 'overall_rating',
 'liquidity_rating',
 'expenses_rating',
 'returns_rating',
 'volatility_rating',
 'dividend_rating',
 'concentration_rating',
 'esg_score',
 'esg_score_peer_percentile',
 'esg_score_global_percentile',
 'carbon_intensity',
 'sri_exclusion_criteria',
 'sustainable_impact_solutions']

In [6]:
data = []

while True:
    try:
        for tr in table.find("tbody").find_all("tr"):
            data.append([td.get_text(strip=True) for td in tr.find_all("td")])
    
        next_button = driver.find_element(By.XPATH, "//li[contains(@class, 'page-next')]/a[contains(text(), 'Next ›')]")

        parent_li = next_button.find_element(By.XPATH, "..")
        if "disabled" in parent_li.get_attribute("class"):
            print("No more pages to navigate.")
            break
            
        next_button.click()
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")        
        table = soup.find("table", id="etfs")

    except:
        print(f"An error occurred")
        break


No more pages to navigate.


In [7]:
len(data)

3118

In [8]:
import pandas as pd
df = pd.DataFrame(data, columns=headers)

In [9]:
df.head(5)

,symbol,etf_name,total_assets,ytd,avg_volume,previous_closing_price,1_day_change,1_week,1_month,1_year,...,returns_rating,volatility_rating,dividend_rating,concentration_rating,esg_score,esg_score_peer_percentile,esg_score_global_percentile,carbon_intensity,sri_exclusion_criteria,sustainable_impact_solutions
0,AADR,AdvisorShares Dorsey WrightADRETF,$44.92,23.10%,"4,100",$84.72,-0.20%,-0.57%,2.78%,35.79%,...,,,,,6.17,0.59%,39.37%,180.63,11.32%,5.92%
1,AALG,Leverage Shares 2X LongAALDailyETF,$0.65,N/A,N/A,$13.16,1.46%,-6.34%,-10.98%,N/A,...,,,,,0.0,N/A,N/A,N/A,N/A,N/A
2,AAPB,GraniteShares 2x LongAAPLDailyETF,$32.16,-21.96%,"394,614",$26.35,-0.94%,6.78%,4.33%,-2.47%,...,,,,,0.0,N/A,N/A,N/A,N/A,N/A
3,AAPD,Direxion DailyAAPLBear 1X SharesETF,$25.24,0.49%,"1,249,364",$15.13,0.40%,-3.26%,-2.58%,-10.66%,...,,,,,5.25,13.64%,22.49%,1.69,0.00%,0.01%
4,AAPR,Innovator Equity Defined ProtectionETF– 2 Yr t...,$58.50,5.73%,"3,803",$27.87,0.14%,0.05%,0.92%,7.03%,...,,,,,0.0,N/A,N/A,N/A,N/A,N/A


In [10]:
blank_columns = [col for col in df.columns if df[col].replace('', pd.NA).isna().all()]

In [11]:
blank_columns

['ytd_ff',
 '1_week_ff',
 '4_week_ff',
 '1_year_ff',
 '3_year_ff',
 '5_year_ff',
 'overall_rating',
 'returns_rating',
 'volatility_rating',
 'dividend_rating',
 'concentration_rating']

In [12]:
df = df.drop(columns=blank_columns)
df.head(5)

,symbol,etf_name,total_assets,ytd,avg_volume,previous_closing_price,1_day_change,1_week,1_month,1_year,...,etf_home_page,head_to_head,liquidity_rating,expenses_rating,esg_score,esg_score_peer_percentile,esg_score_global_percentile,carbon_intensity,sri_exclusion_criteria,sustainable_impact_solutions
0,AADR,AdvisorShares Dorsey WrightADRETF,$44.92,23.10%,"4,100",$84.72,-0.20%,-0.57%,2.78%,35.79%,...,View,View,C+,N/A,6.17,0.59%,39.37%,180.63,11.32%,5.92%
1,AALG,Leverage Shares 2X LongAALDailyETF,$0.65,N/A,N/A,$13.16,1.46%,-6.34%,-10.98%,N/A,...,View,View,N/A,N/A,0.0,N/A,N/A,N/A,N/A,N/A
2,AAPB,GraniteShares 2x LongAAPLDailyETF,$32.16,-21.96%,"394,614",$26.35,-0.94%,6.78%,4.33%,-2.47%,...,View,View,B+,N/A,0.0,N/A,N/A,N/A,N/A,N/A
3,AAPD,Direxion DailyAAPLBear 1X SharesETF,$25.24,0.49%,"1,249,364",$15.13,0.40%,-3.26%,-2.58%,-10.66%,...,View,View,N/A,C,5.25,13.64%,22.49%,1.69,0.00%,0.01%
4,AAPR,Innovator Equity Defined ProtectionETF– 2 Yr t...,$58.50,5.73%,"3,803",$27.87,0.14%,0.05%,0.92%,7.03%,...,View,View,C+,N/A,0.0,N/A,N/A,N/A,N/A,N/A


In [13]:
view_columns = [col for col in df.columns if df[col].replace('View', pd.NA).isna().all()]
view_columns

['complete',
 'advanced',
 'fact_sheet',
 'etf_holdings',
 'chart',
 'etf_home_page',
 'head_to_head']

In [14]:
df = df.drop(columns=view_columns)
df.head(5)

,symbol,etf_name,total_assets,ytd,avg_volume,previous_closing_price,1_day_change,1_week,1_month,1_year,...,resistance_1,rsi,liquidity_rating,expenses_rating,esg_score,esg_score_peer_percentile,esg_score_global_percentile,carbon_intensity,sri_exclusion_criteria,sustainable_impact_solutions
0,AADR,AdvisorShares Dorsey WrightADRETF,$44.92,23.10%,"4,100",$84.72,-0.20%,-0.57%,2.78%,35.79%,...,$85.59,61.47,C+,N/A,6.17,0.59%,39.37%,180.63,11.32%,5.92%
1,AALG,Leverage Shares 2X LongAALDailyETF,$0.65,N/A,N/A,$13.16,1.46%,-6.34%,-10.98%,N/A,...,N/A,46.10,N/A,N/A,0.0,N/A,N/A,N/A,N/A,N/A
2,AAPB,GraniteShares 2x LongAAPLDailyETF,$32.16,-21.96%,"394,614",$26.35,-0.94%,6.78%,4.33%,-2.47%,...,$26.88,59.54,B+,N/A,0.0,N/A,N/A,N/A,N/A,N/A
3,AAPD,Direxion DailyAAPLBear 1X SharesETF,$25.24,0.49%,"1,249,364",$15.13,0.40%,-3.26%,-2.58%,-10.66%,...,$15.25,39.34,N/A,C,5.25,13.64%,22.49%,1.69,0.00%,0.01%
4,AAPR,Innovator Equity Defined ProtectionETF– 2 Yr t...,$58.50,5.73%,"3,803",$27.87,0.14%,0.05%,0.92%,7.03%,...,$27.89,71.51,C+,N/A,0.0,N/A,N/A,N/A,N/A,N/A


In [15]:
df.to_parquet("etf_info.parquet", index=False)

In [16]:
driver.close()

In [17]:
df[df['symbol'] == 'AADR']

,symbol,etf_name,total_assets,ytd,avg_volume,previous_closing_price,1_day_change,1_week,1_month,1_year,...,resistance_1,rsi,liquidity_rating,expenses_rating,esg_score,esg_score_peer_percentile,esg_score_global_percentile,carbon_intensity,sri_exclusion_criteria,sustainable_impact_solutions
0,AADR,AdvisorShares Dorsey WrightADRETF,$44.92,23.10%,"4,100",$84.72,-0.20%,-0.57%,2.78%,35.79%,...,$85.59,61.47,C+,N/A,6.17,0.59%,39.37%,180.63,11.32%,5.92%
